Q2.  Consider a simple N-Queens problem.
  Take a suitable value of N and write a Python code for simulated annealing to solve the problem.  
  As we know, if we take T=0 in SA, it falls back to simple hill climbing search.  
  Modify the SA algorithm in order to make it an HC solution.  
  In the report, write proper theoretical justification in support of your modification (e.g., calculation of the acceptance probability) and the behavioral difference between the two approaches.  
  (Note: No marks will be given for simply implementing the HC solution.)

In [33]:
import random, math,copy
import pandas as pd
# import numpy as np
random.seed(42)
# np.random.seed(42)
# import copy

In [34]:
from IPython.display import display, HTML

def n_queens_to_html_with_conflicts(board, heuristic_dict):
    """
    Generates an HTML table representation of an N-Queens board with conflicts in blank cells.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        heuristic_dict (dict): Dictionary containing conflict information for each cell.

    Returns:
        str: HTML string for the N-Queens board.
    """
    n = len(board)
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .queen { background-color: #f4cccc; font-weight: bold; color: black; }
        .conflict { background-color: #f9f9f9; color: gray; }
    </style>
    <table>
    """

    for row in range(n):
        html += "<tr>"
        for col in range(n):
            conflicts = heuristic_dict[(row, col)]['conflicts']
            if heuristic_dict[(row, col)]['queen']:
                conflicts = conflicts if conflicts> 0 else ''
                # If a queen is present, display 'Q' with number of conflicts
                html += f'<td class="queen">Q {conflicts}</td>'
            else:
                # If no queen, display the number of conflicts
                html += f'<td class="conflict">{conflicts}</td>'
        html += "</tr>"

    html += "</table>"
    return html


In [35]:
def create_heuristic_dict(board, n):
    """
    Creates a heuristic dictionary for the N-Queens board.
    The dictionary shows the total number of conflicts on the board
    if the queen from a specific column is moved to a particular position (row, col).

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).

    Returns:
        dict: A dictionary where each key is a tuple (row, col), and the value is a dictionary with:
              - 'queen': Whether a queen is currently present in the cell (True/False).
              - 'conflicts': Total number of conflicts on the board if the queen is moved to this cell.
    """
    heuristic_dict = {}

    for col in range(n):  # Iterate over each column
        original_row = board[col]  # Current row of the queen in this column
        for row in range(n):  # Iterate over each row in the column
            # Temporarily move the queen to the new position
            temp_board = board[:]
            temp_board[col] = row

            # Calculate the total number of conflicts for the new board
            conflicts = 0
            for queen_a in range(n):
                for queen_b in range(queen_a + 1, n):
                    # Check for conflicts (same row or same diagonal)
                    if temp_board[queen_a] == temp_board[queen_b] or \
                            abs(temp_board[queen_a] - temp_board[queen_b]) == abs(queen_a - queen_b):
                        conflicts += 1

            # Add the cell's data to the heuristic dictionary
            heuristic_dict[(row, col)] = {
                'queen': (original_row == row),  # True if the queen is currently in this cell
                'conflicts': conflicts
            }

    return heuristic_dict

- Heuristic Dictionary:
    - Each key is a tuple (row, col) representing a cell.
    - The value is a dictionary with:
    - 'queen': Whether the queen is currently in this cell.
    - 'conflicts': The total number of conflicts on the board if the queen is moved to this cell.

- Conflict Calculation:
    - For each pair of queens (queen_a, queen_b) on the board, check if they are in conflict:
    - Same row: temp_board[queen_a] == temp_board[queen_b]
    - Same diagonal: abs(temp_board[queen_a] - temp_board[queen_b]) == abs(queen_a - queen_b)


In [36]:
def get_new_board(n = 8):

    # Initialize the board randomly
    # The Queen's position on the board (row, column) =  (index,value) of the list
    # Each index represents a column, and the value at that index represents the row of the queen.

    board = [random.randint(0, n - 1) for i in range(n)]
    heuristic_dict = create_heuristic_dict(board, n)  # Generate the heuristic dictionary

    # Generate the HTML representation
    display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))
    return board, heuristic_dict

In [37]:
n=8
board, heuristic_dict = get_new_board(n = n)
print(board)

9,Q 10,10,10,7,10,10,12
Q 10,12,14,11,10,13,Q 10,Q 10
9,10,11,11,9,Q 10,8,10
9,11,12,Q 10,Q 10,11,9,10
8,9,Q 10,13,9,11,7,10
7,10,12,10,7,9,8,8
9,11,10,8,7,9,7,10
10,9,9,8,6,9,7,9


[1, 0, 4, 3, 3, 2, 1, 1]


In [38]:
def calculate_total_conflicts(board,n):
    """Helper function to calculate the total number of conflicts on the board."""
    heuristic_dict = create_heuristic_dict(board, n)
    total_conflicts = sum(
        heuristic_dict[(row, col)]['conflicts']
        for col in range(n)
        for row in range(n)
        if heuristic_dict[(row, col)]['queen']
    )
    return total_conflicts

- Total Conflict Calculator
    - Uses a generator expression to iterate over all cells in the board.
    - For each cell, If a queen is found, the function retrieves the number of conflicts associated with that cell and includes it in the sum.
    - This ensures that only the conflicts caused by the queens currently on the board are considered.

In [39]:
def hill_climbing_with_heuristic(board, n):
    """
    Hill Climbing algorithm to solve the N-Queens problem using heuristic_dict.
    The algorithm evaluates moving each queen to a different row in its column
    and selects the move that reduces the total number of conflicts.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).

    Returns:
        tuple: A tuple containing the final board configuration and the number of conflicts.
    """

    current_conflicts = calculate_total_conflicts(board,n)

    while current_conflicts > 0:
        best_move = None
        best_conflicts = current_conflicts

        # Try moving each queen to a different row in its column
        for col in range(n):
            original_row = board[col]
            for row in range(n):
                if row == original_row:
                    continue  # Skip the current position of the queen

                # Create a new board with the queen moved
                new_board = board[:]
                new_board[col] = row

                # Calculate the total conflicts for the new board
                new_conflicts = calculate_total_conflicts(new_board,n)

                # If the new board has fewer conflicts, update the best move
                if new_conflicts < best_conflicts:
                    best_move = (col, row)
                    best_conflicts = new_conflicts

        # If no better move is found, terminate (local minimum)
        if best_move is None:
            break

        # Make the best move
        col, row = best_move
        print(f"Moving column {col+1} Queen, from row {board[col]+1} to {row+1}")
        # col, row = best_move
        board[col] = row
        current_conflicts = best_conflicts

        # Display the updated board with conflicts
        heuristic_dict = create_heuristic_dict(board, n)
        display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))

    return board, current_conflicts

In [ ]:
temp_board = board[:]
solution, conflicts = hill_climbing_with_heuristic(temp_board, n)
print("Final Board:", solution)
print("Conflicts:", conflicts)

Moving column 5 Queen, from row 4 to 8


5,Q 6,6,7,7,7,7,7
Q 6,9,9,8,10,10,Q 6,Q 6
5,7,7,7,9,Q 6,5,6
5,7,7,Q 6,10,7,5,5
4,7,Q 6,9,9,7,4,7
3,7,8,7,7,6,5,4
5,7,6,6,7,7,4,5
6,7,6,6,Q 6,7,5,6


Moving column 1 Queen, from row 2 to 6


5,Q 3,3,4,4,5,5,5
6,6,5,4,7,6,Q 3,Q 3
5,4,4,5,6,Q 3,3,4
5,5,4,Q 3,7,4,3,3
4,6,Q 3,5,6,4,2,5
Q 3,6,6,5,4,4,4,3
5,6,3,3,4,3,2,3
6,5,4,3,Q 3,4,2,4


Moving column 7 Queen, from row 2 to 5


4,Q 2,2,3,3,4,5,4
4,4,2,3,5,5,3,Q 2
4,3,2,4,6,Q 2,3,3
4,4,2,Q 2,5,5,3,4
4,6,Q 2,4,6,5,Q 2,6
Q 2,5,3,4,3,5,4,4
4,4,1,2,4,3,2,3
4,4,2,3,Q 2,4,2,4


Moving column 3 Queen, from row 5 to 7


3,Q 1,2,3,2,3,4,2
3,3,2,3,4,3,3,Q 1
2,2,2,4,4,Q 1,4,1
3,2,2,Q 1,4,5,3,2
3,4,2,3,5,3,Q 1,3
Q 1,4,3,4,2,4,4,2
3,4,Q 1,3,3,3,3,2
3,4,2,4,Q 1,2,2,2


Final Board: [5, 0, 6, 3, 7, 2, 4, 1]
Conflicts: 8


- Explanation:
    - The hill climbing with heuristic function will use the create heuristic dict to evaluate moves based on the total number of conflicts.

- Dynamic Conflict Calculation:
    - For each column, iterate over all possible rows where the queen could be moved.
    - Temporarily move the queen to the new position and calculate the total number of conflicts for the entire board.

- Temporary Board:
    - A temporary board is used to simulate moving the queen to a new position without modifying the original board.



In [41]:
def simulated_annealing_with_heuristic(board, n, initial_temperature, cooling_rate):
    """
    Simulated Annealing algorithm to solve the N-Queens problem using heuristic_dict.
    The algorithm evaluates moving each queen to a position one step away in its column
    and probabilistically accepts moves based on the temperature.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).
        initial_temperature (float): Initial temperature for the simulated annealing algorithm.
        cooling_rate (float): Cooling rate for the temperature.

    Returns:
        tuple: A tuple containing the final board configuration and the number of conflicts.
    """
    current_conflicts = calculate_total_conflicts(board, n)
    temperature = initial_temperature
    steps = 0
    while temperature > 0 and current_conflicts > 0:
        moves = []
        for col in range(n):
            current_row = board[col]
            potential_moves = list(range(n))  # All possible rows

            for new_row in potential_moves:
                # Create a new board with the queen moved
                new_board = board[:]
                new_board[col] = new_row

                # Calculate the total conflicts for the new board
                new_conflicts = calculate_total_conflicts(new_board, n)

                # Store the move and its associated conflicts
                moves.append((col, new_row, new_conflicts))

        # Choose a move probabilistically based on the Simulated Annealing acceptance probability
        if not moves:
            break

        # Calculate probabilities for each move
        probabilities = []
        for _, _, new_conflicts in moves:
            delta = new_conflicts - current_conflicts
            if delta < 0:
                probabilities.append(1.0)  # Always accept better moves
            else:
                probabilities.append(math.exp(-delta / temperature))  # Probabilistically accept worse moves

        # Normalize probabilities
        total_probability = sum(probabilities)
        probabilities = [p / total_probability for p in probabilities]

        # Select a move based on the probabilities
        selected_move_index = random.choices(range(len(moves)), weights=probabilities, k=1)[0]
        col, new_row, new_conflicts = moves[selected_move_index]
        steps+=1
        print(f"{steps} \t column {col+1} Queen, row {current_row+1} to {new_row+1}")

        # Apply the selected move
        board[col] = new_row
        current_conflicts = new_conflicts

        # Display the updated board with conflicts
        heuristic_dict = create_heuristic_dict(board, n)
        display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))

        # Cool down the temperature
        temperature *= cooling_rate

    return board, current_conflicts

Explanation:

- Generate All Possible Moves :
    - For each column, iterate over all possible rows where the queen could be moved.
    - Temporarily move the queen to the new position and calculate the total number of conflicts for the entire board.
    - Calculate the total conflicts for each potential move.

- Simulated Annealing Selection:
    - For each move, calculate the acceptance probability based on the Simulated Annealing formula:
    - P = 1 if the move reduces conflicts (delta < 0).
    - P = exp(-delta / temperature) if the move increases conflicts (delta > 0).
    - Normalize the probabilities and use random.choices to select a move probabilistically.

- Apply the Move:
    - Update the board with the selected move and recalculate the total conflicts.

- Visualization:
    - Display the board after each move using n_queens_to_html_with_conflicts.
- Cooling:
    - Reduce the temperature after each iteration to decrease the probability of accepting worse moves over time.

In [42]:
temp_board = board[:]
solution, conflicts = simulated_annealing_with_heuristic(temp_board, n, initial_temperature=100, cooling_rate=0.95)
print("Final Board:", solution)
print("Conflicts:", conflicts)

1 	 column 5 Queen, row 2 to 7


6,Q 7,6,8,7,8,8,8
Q 7,10,9,9,10,11,Q 7,Q 7
7,8,7,8,9,Q 7,6,7
5,9,7,Q 7,10,8,6,7
5,7,Q 7,10,9,8,6,7
4,8,7,9,7,8,5,5
7,9,7,7,Q 7,8,6,7
6,7,5,7,6,8,5,6


2 	 column 1 Queen, row 2 to 2


6,Q 7,6,8,7,8,8,8
Q 7,10,9,9,10,11,Q 7,Q 7
7,8,7,8,9,Q 7,6,7
5,9,7,Q 7,10,8,6,7
5,7,Q 7,10,9,8,6,7
4,8,7,9,7,8,5,5
7,9,7,7,Q 7,8,6,7
6,7,5,7,6,8,5,6


3 	 column 1 Queen, row 2 to 6


...

90 	 column 3 Queen, row 5 to 3


2,Q 1,3,3,2,4,2,3
3,3,3,4,3,2,Q 1,3
1,3,Q 1,4,2,4,3,4
1,3,4,3,3,Q 1,4,3
3,3,2,4,3,4,4,Q 1
Q 1,4,4,4,Q 1,5,5,4
0,3,3,2,1,3,2,2
2,3,4,Q 1,2,2,3,3


91 	 column 1 Queen, row 5 to 7


2,Q,2,2,2,2,2,2
3,2,2,3,2,2,Q,2
1,2,Q,2,3,3,2,3
1,2,2,3,3,Q,3,2
3,1,2,3,3,3,3,Q
1,3,2,2,Q,3,3,2
Q,2,3,2,2,3,2,2
2,3,2,Q,2,1,2,2


Final Board: [6, 0, 2, 7, 5, 3, 1, 4]
Conflicts: 0
